In [1]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))  # Only if device_count > 0

train_loader = torch.load('train_dataloader_aug.pth', weights_only=False)
test_loader = torch.load('test_dataloader_aug.pth', weights_only=False)

True
1
NVIDIA GeForce RTX 2060


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmotionRecognition(nn.Module):
    def __init__(self):
        super(EmotionRecognition, self).__init__()

        # Convolutional Block 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # Convolutional Block 2
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)

        # Dummy input to determine flatten size after conv layers
        with torch.no_grad():
            dummy = torch.zeros(1, 3, 48, 48)
            dummy = self._forward_conv(dummy)
            self.flattened_size = dummy.view(1, -1).size(1)

        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.fc2 = nn.Linear(512, 7)

    def _forward_conv(self, x):
        # Only the convolutional part, used for shape inference and forward
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [3]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        alpha: tensor of shape (num_classes,) for class weights
        gamma: focusing parameter
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # class weights
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)  # pt = probability of the true class
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [4]:
import torch
import torch.optim as optim
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
if device.type == "cuda":
    torch.cuda.manual_seed(42)
print(f"Using device: {device}")
model = EmotionRecognition().to(device)
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn

class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_loader.dataset.targets),
                                     y=train_loader.dataset.targets)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = FocalLoss(alpha=class_weights_tensor, gamma=1.5)
optimizer = optim.Adam(model.parameters(), lr=0.002, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

scaler = torch.cuda.amp.GradScaler() if device.type == "cuda" else None



Using device: cuda


C:\Users\Korisnik\AppData\Local\Temp\ipykernel_10624\1828967484.py:25: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if device.type == "cuda" else None


In [5]:
def train_one_epoch(model, loader, optimizer, criterion, scaler):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0

    for images, labels in tqdm(loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast(enabled=scaler is not None):
            outputs = model(images)
            loss = criterion(outputs, labels)

        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    return total_loss / total_samples, 100. * total_correct / total_samples

def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_labels), np.array(all_preds)

In [7]:
best_loss = float('inf')
patience = 10
epochs_no_improve = 0
num_epochs = 100
save_path = "best_model.pth"

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, scaler)
    labels, preds = evaluate(model, test_loader)
    test_acc = 100. * np.sum(preds == labels) / len(labels)

    scheduler.step()

    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}%")
    print(f"Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

    # Early stopping
    if train_loss < best_loss:
        best_loss = train_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), save_path)
        print("✅ Saved new best model.")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("⏹ Early stopping triggered.")
            break

Training:   0%|          | 0/449 [00:00<?, ?it/s]C:\Users\Korisnik\AppData\Local\Temp\ipykernel_10624\4020488208.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):



Epoch 1/100
Train Loss: 1.4973, Train Acc: 18.12% | Test Acc: 21.82%
Learning Rate: 0.002000
✅ Saved new best model.



Epoch 2/100
Train Loss: 1.4587, Train Acc: 20.50% | Test Acc: 22.46%
Learning Rate: 0.002000
✅ Saved new best model.



Epoch 3/100
Train Loss: 1.4428, Train Acc: 20.37% | Test Acc: 24.37%
Learning Rate: 0.002000
✅ Saved new best model.



Epoch 4/100
Train Loss: 1.4284, Train Acc: 18.65% | Test Acc: 17.05%
Learning Rate: 0.002000
✅ Saved new best model.


...


Epoch 96/100
Train Loss: 0.5056, Train Acc: 63.49% | Test Acc: 59.53%
Learning Rate: 0.000004



Epoch 97/100
Train Loss: 0.5129, Train Acc: 62.84% | Test Acc: 59.38%
Learning Rate: 0.000004



Epoch 98/100
Train Loss: 0.5040, Train Acc: 63.29% | Test Acc: 59.50%
Learning Rate: 0.000004
✅ Saved new best model.



Epoch 99/100
Train Loss: 0.5053, Train Acc: 63.12% | Test Acc: 59.47%
Learning Rate: 0.000004



Epoch 100/100
Train Loss: 0.5109, Train Acc: 63.37% | Test Acc: 59.33%
Learning Rate: 0.000002


In [8]:
from sklearn.metrics import classification_report, confusion_matrix

# Final evaluation
model.load_state_dict(torch.load(save_path))
model.eval()
labels, preds = evaluate(model, test_loader)

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print("\nClassification Report:")
print(classification_report(labels, preds, target_names=emotion_labels))
print("Confusion Matrix:")
print(confusion_matrix(labels, preds))


Classification Report:
              precision    recall  f1-score   support

       Angry       0.49      0.54      0.51       958
     Disgust       0.59      0.68      0.63       111
        Fear       0.42      0.36      0.39      1024
       Happy       0.87      0.73      0.80      1774
         Sad       0.55      0.60      0.57      1233
    Surprise       0.45      0.47      0.46      1247
     Neutral       0.70      0.82      0.76       831

    accuracy                           0.60      7178
   macro avg       0.58      0.60      0.59      7178
weighted avg       0.60      0.60      0.60      7178

Confusion Matrix:
[[ 519   22  116   33   91  147   30]
 [  19   75    5    2    2    7    1]
 [ 145    6  369   24  125  218  137]
 [ 101    3   55 1300  145  105   65]
 [  95    7   79   61  739  223   29]
 [ 161   10  188   45  232  587   24]
 [  21    4   63   25   16   20  682]]
